## 6.5. 汇聚层

通常当处理图像时，希望逐渐降低隐藏表示的空间分辨率、聚集信息，这样随着在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。

机器学习任务通常会跟全局图像的问题有关。所以希望最后一层的神经元能够对整个输入图像是全局敏感的。

现实中，拍摄照片时，同一时间地点角度拍摄同一个物体都几乎无法保证前后两次拍摄能完美使目标物体保持在完全相同的像素位置

故，引入汇聚层/池化层 pooling的目的是：

1. 降低卷积层对位置的敏感性
2. 降低对空间降采样表示的敏感性